In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV
import time
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
)
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPClassifier


#Importing the functions created in main.py
from Preprocessing_functions2 import *
import importlib
imported_module = importlib.import_module("Preprocessing_functions2")
importlib.reload(imported_module)

# pandas max columns display
pd.set_option('display.max_columns', None)

## Import Dataset

In [3]:
train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')
test_data = pd.read_csv('test_data.csv', index_col='Claim Identifier')

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.csv'

In [ ]:
train_data = train_data[~(train_data.drop(columns=['Assembly Date']).isna().all(axis=1) & train_data['Assembly Date'].notna())]

In [145]:
X = train_data.drop(columns=['Claim Injury Type', 'WCB Decision', 'Agreement Reached','OIICS Nature of Injury Description'])
y = train_data['Claim Injury Type']

test_data = test_data.drop(columns=['OIICS Nature of Injury Description'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

____

# Auxiliary Functions


## Preprocess Functions

In [146]:
CODE_COLUMNS = ['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

DESCRIPTION_COLUMNS = ['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description']

BOOLEAN_COLUMNS = ['Alternative Dispute Resolution', 'Attorney/Representative','COVID-19 Indicator']

date_order = ['Accident Date', 'C-2 Date','C-3 Date','Assembly Date', 'First Hearing Date']

numerical_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year', 
    'C-2 Date', 
    'C-3 Date', 
    'First Hearing Date', 
    'IME-4 Count', 
]

outliers_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year',
    'IME-4 Count', 
]

categorical_features = ['Alternative Dispute Resolution',
 'Attorney/Representative',
 'Carrier Name',
 'Carrier Type',
 'County of Injury',
 'COVID-19 Indicator',
 'District Name',
 'Gender',
 'Industry Code',
 'Medical Fee Region',
 'WCIO Cause of Injury Code',
 'WCIO Nature of Injury Code',
 'WCIO Part Of Body Code',
 'Zip Code']


columns_to_scale = ['Accident Date',
                'Assembly Date',
                'Average Weekly Wage',
                'Age at Injury',
                'Birth Year', 
                'Number of Dependents',
                'IME-4 Count']

date_columns = ['Accident Date', 'Assembly Date']

columns_to_drop = ['C-2 Date', 'C-3 Date', 'First Hearing Date']

low_cardinality_cols = [col for col in categorical_features if X[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_features if X[col].nunique() > 10]


In [147]:
binning2_columns = ['Age at Injury', 'Age at Injury', 'Birth Year', 'Average Weekly Wage', 'IME-4 Count']
date_columns = ['Accident Date', 'Assembly Date']

def create_groupingFeatures(X_train, X_val):

    categorical_features = []
    numerical_features = []

    X_train, X_val= newFeature_binnedGroups(X_train, X_val, binning2_columns, 6)
    # Add all names of the binning_column + 'Group' to the list of categorical_features
    for col in binning2_columns:
        categorical_features.append(col + ' Group')
        X_train[f'{col} Group'].astype(str)
        X_val[f'{col} Group'].astype(str)

    X_train, X_val = newFeature_month(X_train, X_val, date_columns)
    # Add all names of the date_columns + 'Month' to the list of categorical_features
    for col in date_columns:
        categorical_features.append(col + ' Month')
        X_train[f'{col} Month'].astype(str)
        X_val[f'{col} Month'].astype(str)



    X_train, X_val = newFeature_daysBetween(X_train, X_val, firstDate='Accident Date', secondDate='Assembly Date')
    numerical_features.append('Days Between Accident Date and Assembly Date')
    
    return X_train, X_val, categorical_features, numerical_features

In [148]:
def preprocessing_scaling_encoding_dum(X_train, X_val):
    X_train, X_val = type_conversion_categorical(X_train, X_val,categorical_features)
    X_train, X_val = drop_description_columns(X_train, X_val)
    X_train, X_val = convert_to_timestamp(X_train, X_val, date_order)
    X_train, X_val = convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    X_train, X_val = impute_mean_numerical(X_train, X_val, numerical_columns)
    X_train, X_val = fill_missing_with_mode(X_train, X_val)
    X_train, X_val = feature_creation_has_Cdate(X_train, X_val)
    X_train, X_val = drop_unwanted_columns(X_train, X_val, columns_to_drop)
    X_train, X_val = winsorize_outliers(X_train, X_val, outliers_columns)
    X_train, X_val = scaling_robust(X_train, X_val, columns_to_scale)
    X_train, X_val = encoding_onehot(X_train, X_val, low_cardinality_cols)
    X_train, X_val = encoding_frequency1(X_train, X_val, high_cardinality_cols)

    return X_train, X_val

In [149]:
def preprocessing_newFeatures_advanced(X_train, X_val):

    # Type conversion
    X_train, X_val = type_conversion_categorical(X_train, X_val, categorical_features)
    X_train, X_val = convert_to_timestamp(X_train, X_val, date_order)
    X_train, X_val = convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)

    

    # Knowledge-based imputation of features
    X_train, X_val = fill_missing_codes_description_based(X_train, X_val)
    X_train, X_val = fillna_zip_code(X_train, X_val)
    X_train, X_val = fillnan_accident_date(X_train, X_val)
    X_train, X_val = fillnan_birth_year(X_train, X_val)
    X_train, X_val = impute_weekly_wage_with_zipIndustryCode(X_train, X_val)
    X_train, X_val = fillnan_IME4_count(X_train, X_val)

    # Impute still missing values
    X_train, X_val = impute_mean_numerical(X_train, X_val, numerical_columns)
    X_train, X_val = fill_missing_with_mode(X_train, X_val)

    

    # Feature creation
    X_train, X_val = feature_creation_has_Cdate(X_train, X_val)
    X_train, X_val = drop_unwanted_columns(X_train, X_val, columns_to_drop)
    X_train, X_val = newFeature_hasIME4(X_train, X_val)
    X_train, X_val = drop_description_columns(X_train, X_val)
    #X_train, X_val = convert_to_datetime(X_train, X_val, date_columns)

    # # Grouping features
    # X_train, X_val, c_features, n_features= create_groupingFeatures(X_train, X_val)
    # categorical_features.append(c_features)
    # numerical_columns.append(n_features)

    # Redo type conversion
    #X_train, X_val = type_conversion_categorical(X_train, X_val, categorical_features)
    #X_train, X_val = convert_to_timestamp(X_train, X_val, date_columns)

    # Treating outliers
    X_train, X_val = winsorize_outliers(X_train, X_val, outliers_columns)

    # Scaling
    X_train, X_val = scaling_robust(X_train, X_val, binning2_columns)
    X_train, X_val = scaling_robust(X_train, X_val, date_columns)

    # # Ensure categorical_features contains unique column names
    # valid_categorical_features = list(set(categorical_features))

    # # Verify all columns in categorical_features exist in X_train
    # valid_categorical_features = [col for col in valid_categorical_features if col in X_train.columns]
    # print(f'Valid categorical features: {valid_categorical_features}')


    low_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() < 15]
    high_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() > 15]

    X_train, X_val = encoding_onehot(X_train, X_val, low_cardinality_cols)
    X_train, X_val = encoding_frequency1(X_train, X_val, high_cardinality_cols)

    return X_train, X_val

# Feature Selection


## lasso regresion 

In [ ]:
X_train, X_val = preprocessing_scaling_encoding_dum(X_train, X_val)

In [ ]:

logreg_cv = LogisticRegressionCV(
    penalty='l1',
    solver='liblinear',
    Cs=10,  # Number of regularization strengths to test
    cv=5,   # 5-fold cross-validation
    random_state=42
)
logreg_cv.fit(X_train, y_train)

# Identify selected and unselected features
coefs = logreg_cv.coef_.flatten()
selected_features = X.columns[coefs != 0].tolist()
unselected_features = X.columns[coefs == 0].tolist()

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.bar(X.columns, coefs, color=["green" if coef != 0 else "red" for coef in coefs])
plt.xlabel("Features")
plt.ylabel("Coefficient Value")
plt.title("Feature Importance via Logistic Regression with L1 Penalty")
plt.axhline(0, color="black", linewidth=0.8, linestyle="--")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Output selected and unselected features
print("Selected Features:", selected_features)
print("Unselected Features:", unselected_features)


## RFECV with preprocessing_scaling_encoding_dum

In [ ]:
X_train, X_val = preprocessing_scaling_encoding_dum(X_train, X_val)
y_train, y_val = encoding_label(y_train, y_val) 

In [ ]:
 # Initialize RandomForest model
rf_model = RandomForestClassifier(random_state=42)

# Set up cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up RFECV with RandomForest and cross-validation
rfecv = RFECV(estimator=rf_model, step=1, cv=cv_strategy, scoring='f1_macro') 

# Fit RFECV
rfecv.fit(X_train, y_train)

#Get the selected features
selected_features_RF = X_train.columns[rfecv.support_].tolist()
optimal_num_features = rfecv.n_features_

print("Optimal number of features:", optimal_num_features)
print("Selected Features:", selected_features_RF)

In [152]:
pptimal_num_features = 46
selected_features = ['Accident Date', 'Age at Injury', 'Assembly Date', 'Average Weekly Wage', 'Birth Year', 'Number of Dependents', 'Has First Hearing Date', 'Alternative Dispute Resolution_False', 'Alternative Dispute Resolution_True', 'Alternative Dispute Resolution_nan', 'Attorney/Representative_False', 'Attorney/Representative_True', 'Carrier Type_1A. PRIVATE', 'Carrier Type_2A. SIF', 'Carrier Type_3A. SELF PUBLIC', 'Carrier Type_4A. SELF PRIVATE', 'Carrier Type_5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)', 'Carrier Type_5C. SPECIAL FUND - POI CARRIER WCB MENANDS', 'Carrier Type_5D. SPECIAL FUND - UNKNOWN', 'Carrier Type_UNKNOWN', 'COVID-19 Indicator_False', 'COVID-19 Indicator_True', 'District Name_ALBANY', 'District Name_BINGHAMTON', 'District Name_BUFFALO', 'District Name_HAUPPAUGE', 'District Name_NYC', 'District Name_ROCHESTER', 'District Name_STATEWIDE', 'District Name_SYRACUSE', 'Gender_F', 'Gender_M', 'Gender_U', 'Gender_X', 'Medical Fee Region_I', 'Medical Fee Region_II', 'Medical Fee Region_III', 'Medical Fee Region_IV', 'Medical Fee Region_UK', 'Carrier Name', 'County of Injury', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Zip Code']

## RFECV with preprocessing_newFeatures_advanced

In [153]:
X_train, X_val = preprocessing_newFeatures_advanced(X_train, X_val)
y_train, y_val = encoding_label(y_train, y_val) 

c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:619: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Accident Date'].fillna(X_train['C-2 Date'] - mean_difference_c2_accident, inplace=True)
c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:620: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The beh

In [155]:
 # Initialize RandomForest model
rf_model = RandomForestClassifier(random_state=42)

# Set up cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5)

# Set up RFECV with RandomForest and cross-validation
rfecv = RFECV(estimator=rf_model, step=1, cv=cv_strategy, scoring='f1_macro', n_jobs=-1) 

# Fit RFECV
rfecv.fit(X_train, y_train)

#Get the selected features
selected_features_RF = X_train.columns[rfecv.support_].tolist()
feature_ranking = rfecv.ranking_
optimal_num_features = rfecv.n_features_

print("Optimal number of features:", optimal_num_features)
print("Feature Ranking:", feature_ranking)
print("Selected Features:", selected_features_RF)

Optimal number of features: 38
Feature Ranking: [ 1  1  1  1  1 13  1 12 11 10  9  1  3  8  1  1  1  1  1  1  7  6  4  1
  1  1  1  1  1  1  1  1  1  1  1  1  2  5  1  1  1  1  1  1  1  1  1  1
  1  1]
Selected Features: ['Accident Date', 'Age at Injury', 'Assembly Date', 'Average Weekly Wage', 'Birth Year', 'Number of Dependents', 'Alternative Dispute Resolution_False', 'Attorney/Representative_False', 'Attorney/Representative_True', 'Carrier Type_1A. PRIVATE', 'Carrier Type_2A. SIF', 'Carrier Type_3A. SELF PUBLIC', 'Carrier Type_4A. SELF PRIVATE', 'Carrier Type_UNKNOWN', 'COVID-19 Indicator_False', 'COVID-19 Indicator_True', 'District Name_ALBANY', 'District Name_BINGHAMTON', 'District Name_BUFFALO', 'District Name_HAUPPAUGE', 'District Name_NYC', 'District Name_ROCHESTER', 'District Name_STATEWIDE', 'District Name_SYRACUSE', 'Gender_F', 'Gender_M', 'Medical Fee Region_I', 'Medical Fee Region_II', 'Medical Fee Region_III', 'Medical Fee Region_IV', 'Medical Fee Region_UK', 'Carrier Na

In [157]:
#create the feature ranking wee the features and the ranking are together
feature_ranking = pd.DataFrame({'Feature': X_train.columns, 'Ranking': feature_ranking})
optimal_num_features_advanced = 46
selected_featuer_advanced_rf = ['Accident Date', 'Age at Injury', 'Assembly Date', 'Average Weekly Wage', 'Birth Year', 'Number of Dependents', 'Alternative Dispute Resolution_False', 'Attorney/Representative_False', 'Attorney/Representative_True', 'Carrier Type_1A. PRIVATE', 'Carrier Type_2A. SIF', 'Carrier Type_3A. SELF PUBLIC', 'Carrier Type_4A. SELF PRIVATE', 'Carrier Type_UNKNOWN', 'COVID-19 Indicator_False', 'COVID-19 Indicator_True', 'District Name_ALBANY', 'District Name_BINGHAMTON', 'District Name_BUFFALO', 'District Name_HAUPPAUGE', 'District Name_NYC', 'District Name_ROCHESTER', 'District Name_STATEWIDE', 'District Name_SYRACUSE', 'Gender_F', 'Gender_M', 'Medical Fee Region_I', 'Medical Fee Region_II', 'Medical Fee Region_III', 'Medical Fee Region_IV', 'Medical Fee Region_UK', 'Carrier Name', 'County of Injury', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Zip Code']

In [158]:
feature_ranking

,Feature,Ranking
0,Accident Date,1
1,Age at Injury,1
2,Assembly Date,1
3,Average Weekly Wage,1
4,Birth Year,1
5,IME-4 Count,13
6,Number of Dependents,1
7,Has C-3 Date,12
8,Has C-2 Date,11
9,Has First Hearing Date,10


## GridSearchCV Performance Evaluation


In [21]:
def create_predifined_split(X, y, preprocess_steps,selected_features, n_splits = 5):
    """
    Creates a PredefinedSplit object to be used in cross-validation, more specifically in GridSearchCV.

    Steps:
    - Defines the number of splits
    - Splits the data into training and validation sets
    - Applies the preprocessing steps to the training and validation sets
    - Returns the PredefinedSplit object and the preprocessed data
    """

    X_combined_list = []
    y_combined_list = []

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    test_data = np.zeros(len(X), dtype=int) - 1

    for fold_idx, (_, test_idx) in enumerate(kf.split(X, y)):
        test_data[test_idx] = fold_idx

    ps = PredefinedSplit(test_fold=test_data)

    for train_index, test_index in ps.split():

        # Get fold
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

        # Preprocess and encode data    
        X_train, X_val = preprocess_steps(X_train, X_val)
        y_train, y_val = encoding_label(y_train, y_val)

        X_combined_list.append(X_train[selected_features])
        y_combined_list.append(y_train)

    X_combined = pd.concat(X_combined_list, axis=0)
    y_combined = np.concatenate(y_combined_list, axis=0)

    return ps, X_combined, y_combined


        


In [22]:
def get_best_parameters(X, y, model, param_grid, preprocess_steps, n_splits=5):
    """
    Finds the best hyperparameters for a given model using GridSearchCV.

    Steps:
    - Creates a PredefinedSplit object
    - Creates a GridSearchCV object
    - Fits the GridSearchCV object
    - Returns the best hyperparameters and the best score
    """
    predefined_split, X_combined, y_combined = create_predifined_split(X, y, preprocess_steps,selected_features, n_splits=n_splits)

    scoring = make_scorer(f1_score, average='macro')

    grid_search = GridSearchCV(
        model,
        param_grid,
        cv=predefined_split,
        scoring=scoring,
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X_combined, y_combined)

    print("Best Parameters:", grid_search.best_params_)
    print("Best F1-macro Score:", grid_search.best_score_)

    return grid_search.best_params_, grid_search.best_score_

______

# Model Assessment

## Logistic Regression

## Random Forest

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}
model=RandomForestClassifier()

In [ ]:
RandomForestClassifier_best_param, RandomForestClassifier_best_score = get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum, n_splits=2)

## NN

In [ ]:
model = MLPClassifier(
        solver='adam',
        max_iter=1000,  # Increase if needed
        random_state=42,
    )

In [ ]:
param_grid = {
    'hidden_layer_sizes': [
        (int(0.75 * len(selected_features)), int(0.5 * len(selected_features))),  # Original configuration
        (int(0.5 * len(selected_features)), int(0.25 * len(selected_features)), int(0.125 * len(selected_features))),  # Three layers
    ],
    'learning_rate_init': [0.01, 0.1],  # Test lower and higher learning rates
    'activation': ['relu', 'tanh'],  # Compare relu and tanh
    'alpha': [0.001, 0.01],  # Regularization strength
    'batch_size': ['auto', 64, 128],  # Test different batch sizes
}

In [ ]:
# nnGS_best_params, nnGS_best_score = get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum, selected_features, n_splits=5)

Best Parameters: {'activation': 'tanh', 'alpha': 0.001, 'batch_size': 'auto', 'hidden_layer_sizes': (34, 23), 'learning_rate_init': 0.01}<br>
Best F1-macro Score: 0.30394956984435917

In [ ]:
model_NN = MLPClassifier(
            solver='adam',
            activation='tanh',
            alpha=0.001,
            learning_rate_init=0.01,
            hidden_layer_sizes=(int(0.75 * len(selected_features)), int(0.5 * len(selected_features))),
            max_iter=1000, 
            random_state=42,
            batch_size='auto'
            )

In [ ]:
# X, test_data = preprocessing_scaling_encoding_dum(X, test_data)

c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:403: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:404: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace metho

In [ ]:
# le = LabelEncoder()
# y = le.fit_transform(y)

In [ ]:
# model_NN.fit(X[selected_features], y)

# predict_test = model_NN.predict(test_data[selected_features])

In [ ]:
# predict_test = le.inverse_transform(predict_test)

# XGBoost

In [13]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 1, 10],
    'reg_lambda': [1, 10, 100]
}
model=XGBClassifier(random_state=42)

In [ ]:
get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum, n_splits=5)

## Knn

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform'], # Weight function
    'metric': ['minkowski'],
    'p': [1, 2], # Power parameter for Minkowski distance
    'algorithm': ['auto', 'ball_tree'],  # Algorithm for nearest neighbor search
    'leaf_size': [20, 30, 40, 50] 
}

model=KNeighborsClassifier()

In [32]:
knn_best_param, knn_best_score = get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum, n_splits=2)

/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions2.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[coulmns] = X_train[coulmns].astype(str)
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions2.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[coulmns] = X_val[coulmns].astype(str)
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preproces

Fitting 2 folds for each of 128 candidates, totalling 256 fits


KeyboardInterrupt: 

## Ensamble Models
look for last year project


# Final Model